In [2]:
def count_parameters_conv(in_channels: int, out_channels: int, kernel_size: int, bias: bool):
    return (in_channels * (kernel_size ** 2) + bias * 1) * out_channels

In [1]:
True * 1

1

In [12]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as T
from IPython.display import clear_output
from PIL import Image
from matplotlib import cm
from time import perf_counter
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from tqdm import tqdm

warnings.filterwarnings('ignore')

plt.rc('font', size=30)

In [11]:
mnist_train = MNIST(
"C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/mnist",
train=True,
download=True,
transform=T.ToTensor()
)

mnist_valid = MNIST(
"C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/mnist",
train=False,
download=True,
transform=T.ToTensor()
)

Failed to download (trying next):
HTTP Error 403: Forbidden




Extracting C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/mnist\MNIST\raw\train-images-idx3-ubyte.gz to C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/mnist\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden




Extracting C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/mnist\MNIST\raw\train-labels-idx1-ubyte.gz to C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/mnist\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden




Extracting C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/mnist\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden




Extracting C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/mnist\MNIST\raw



NameError: name 'DataLoader' is not defined

In [13]:
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
valid_loader = DataLoader(mnist_valid, batch_size=64, shuffle=False)

In [14]:
def train(model: nn.Module) -> float:
    model.train()

    train_loss = 0

    for x, y in tqdm(train_loader, desc='Train'):
        optimizer.zero_grad()

        output = model(x)

        loss = loss_fn(output, y)

        train_loss += loss.item()

        loss.backward()

        optimizer.step()

    train_loss /= len(train_loader)
    
    return train_loss

In [17]:
@torch.inference_mode()
def evaluate(model: nn.Module, loader: DataLoader):
    model.eval()

    total_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(loader, desc='Evaluation'):
        output = model(x)

        loss = loss_fn(output, y)

        total_loss += loss.item()

        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()

    total_loss /= len(loader)
    accuracy = correct / total

    return total_loss, accuracy

In [18]:
first_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28 * 28, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

optimizer = torch.optim.Adam(first_model.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()

In [19]:
num_epochs = 15

train_loss_history, valid_loss_history = [], []
valid_accuracy_history = []

start = perf_counter()

for epoch in range(num_epochs):
    train_loss = train(first_model)

    valid_loss, valid_accuracy = evaluate(first_model, valid_loader)

    train_loss_history.append(train_loss)
    valid_loss_history.append(valid_loss)
    valid_accuracy_history.append(valid_accuracy)
    
    clear_output()
    
print(f'Total training and evaluation time {perf_counter() - start:.5f}')

Total training and evaluation time 159.91198


In [20]:
valid_loss_history

[0.15509032216338312,
 0.1134034944446105,
 0.08504324085390563,
 0.07751730395181425,
 0.07141904898190271,
 0.06803332141375876,
 0.06899651843221387,
 0.06782565657245451,
 0.06389782048300864,
 0.06873688355520992,
 0.07812249859455785,
 0.07371480627148375,
 0.07384528461101188,
 0.07692872570440529,
 0.08424550748077164]

In [21]:
valid_accuracy_history

[0.9557,
 0.9643,
 0.9737,
 0.9762,
 0.9763,
 0.9777,
 0.98,
 0.9787,
 0.9803,
 0.9806,
 0.9763,
 0.9784,
 0.98,
 0.9803,
 0.9809]

In [23]:
torch.save(first_model.state_dict(), "C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/model_weights.pt")

In [24]:
def create_mlp_model():
    return nn.Sequential(
    nn.Flatten(),
    nn.Linear(28 * 28, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

In [25]:
create_mlp_model()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=10, bias=True)
)

In [26]:
second_model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),

    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),

    nn.Flatten(),
    nn.Linear(4 * 4 * 64, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

optimizer = torch.optim.Adam(second_model.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()

In [27]:
num_epochs = 15

train_loss_history, valid_loss_history = [], []
valid_accuracy_history = []

start = perf_counter()

for epoch in range(num_epochs):
    train_loss = train(second_model)

    valid_loss, valid_accuracy = evaluate(second_model, valid_loader)

    train_loss_history.append(train_loss)
    valid_loss_history.append(valid_loss)
    valid_accuracy_history.append(valid_accuracy)

    clear_output()

print(f'Total training and evaluation time {perf_counter() - start:.5f}')

Total training and evaluation time 654.82208


In [ ]:
def create_conv_model():
    return nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),

    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),

    nn.Flatten(),
    nn.Linear(4 * 4 * 64, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
    )

In [28]:
torch.save(second_model.state_dict(), "C:/Users/maslo/Desktop/karpov_start_ml/3_module/3_lesson/second_model_weights.pt")

In [29]:
valid_accuracy_history

[0.9833,
 0.9888,
 0.9889,
 0.9928,
 0.9916,
 0.9907,
 0.9919,
 0.9907,
 0.9921,
 0.9915,
 0.9918,
 0.9892,
 0.9926,
 0.9926,
 0.9941]